In [1]:
%%capture
import os

# check if this is a colab notebook and clone the repo if it is
if 'COLAB_GPU' in os.environ:
    %cd /content/
    !git clone https://github.com/Enver-group/twitch-web-analytics
    %cd twitch-web-analytics
else:
    os.chdir('..')

!pip install -r requirements.txt

!pip install -e .

In [2]:
#autoreload modules
%load_ext autoreload
%autoreload 2

from src.user import User

In [3]:
ibai = User.from_name("ibai")
ibai_follows = ibai.follows
print(f"{ibai.name.capitalize()} follows {len(ibai_follows)} users")

Ibai follows 155 users


In [4]:
users_that_ibai_follows = User.get_users(ibai_follows)
print("%s users retrieved from the list of follows"%len(users_that_ibai_follows))

155 users retrieved from the list of follows


In [5]:
import pandas as pd
df = pd.DataFrame([ibai]+users_that_ibai_follows)
df

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,None,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286990597,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"[35980866, 145908612, 205218019, 91136321, 524..."
1,35980866,Perxitaa,None,partner,Troll de Youtube http://www.youtube.com/user/p...,es,Grand Theft Auto V,38776933,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-05T22:12:47Z,None
2,178203816,aroyitt,None,partner,"🦁Creadora de contenido de MAD Lions. 🦁Juego, c...",es,Just Chatting,21303157,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-10-18T12:47:29Z,None
3,62638609,LakshartNia,None,partner,۞ ¸.•*¨*`•.¸ ☆ †Bienvenid@s† ☆ ¸.•*¨*`•.¸ ۞ Ho...,es,Just Chatting,18316428,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-05-15T09:25:11Z,None
4,121606712,3gerardpique,None,partner,FC Barcelona football player | Kosmos Presiden...,es,Sports,504977,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-04-12T13:00:54Z,None
...,...,...,...,...,...,...,...,...,...,...,...
151,102404582,silerr8,None,partner,Midlaner de TEAM QUESO y campeon del SOLOQCHAL...,es,League of Legends,2940632,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-09-17T18:41:42Z,None
152,1895664,DjMaRiiO,None,,http://www.youtube.com/DjMaRiiO,es,FIFA 21,3645746,https://static-cdn.jtvnw.net/jtv_user_pictures...,2008-09-24T21:06:33Z,None
153,22346597,LVPes,None,partner,,es,Special Events,227606609,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,None
154,42028111,LVPes3,None,partner,"Canal de la LVP, Liga de Videojuegos Profesion...",es,League of Legends: Wild Rift,24677182,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-04-03T10:53:28Z,None


In [6]:
from src.data import make_data_from_root_user

# Obtiene datos de cada usuario del arbol empezando por Ibai y sus follows
# cada iteracion siguiente coge un usuario del arbol aleatorio para añadir sus follows 
# y asi hasta llegar a max_users o que se interrumpa manualmente. 
# Cada 50 iteraciones guarda el progreso actual en un archivo csv
make_data_from_root_user("ibai",output_file="data/data.csv",max_users=200)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: ibai
INFO:src.data.make_dataset:New users will be fetched until 200 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 140 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:max_users reached. 1210 users were retrieved.
INFO:src.data.make_dataset:writing dataset to file before stopping...


,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,None,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286990597,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"[35980866, 145908612, 205218019, 91136321, 524..."
1,567658286,TELLIER50,None,partner,"Me llamo Robertson, tengo 71 años y soy jubila...",es,League of Legends,2889051,https://static-cdn.jtvnw.net/jtv_user_pictures...,2020-08-15T17:00:57Z,"[35685492, 211682926, 546281229, 549926115, 59..."
2,29847318,Aitorek,None,partner,Bicampeón de Europa de Fall Guys en Twitch Riv...,es,Fall Guys: Ultimate Knockout,8210153,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-04-17T15:29:40Z,None
3,41251024,dificult,None,affiliate,Player of League Of Legends Challenger Latino ...,es,Axie Infinity,147404,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-03-12T01:55:59Z,None
4,424249007,SoyScarletNatsuno,None,affiliate,💜 Holis chulis! Soy Scarlet y la vida me da pa...,es,Just Chatting,327960,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-03-18T02:39:10Z,None
...,...,...,...,...,...,...,...,...,...,...,...
1205,107627864,SoyFelipez360,None,partner,A veces juego a cosas,es,Minecraft,14705879,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-11-21T13:43:46Z,None
1206,110225525,fixlol1,None,partner,ADC Challenger LAS/BR Redes sociales:-www.face...,es,League of Legends,1142199,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-12-23T00:55:06Z,None
1207,469945659,MasKeChino,None,,Not a GamerTekno 420,es,League of Legends,201,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-10-30T03:09:12Z,None
1208,411478436,ponyito_,None,affiliate,ª,es,osu!,4015,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-01-22T19:08:38Z,None


In [7]:
data = pd.read_csv("data/data.csv")
data.drop_duplicates("id", keep='first',inplace=True)
data.fillna({"broadcaster_type":"user"},inplace=True)
data = data.loc[data['lang']=='es']

In [8]:
data.sort_values(by='view_count', ascending=False)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,NaN,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286990597,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
287,22346597,LVPes,NaN,partner,NaN,es,Special Events,227606609,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,NaN
328,39276140,Rubius,NaN,partner,se contar hasta patata,es,Minecraft,222783415,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12T18:46:56Z,NaN
407,459331509,auronplay,NaN,partner,feliz pero no mucho,es,Minecraft,212157750,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,NaN
7,48878319,TheGrefg,NaN,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Fortnite,197764594,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12T00:45:27Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...
743,452929850,elenaaa_25,NaN,user,NaN,es,League of Legends,28,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-08-06T17:07:39Z,NaN
823,184090500,staurastrum,NaN,user,"Veterano de las arcades principalmente KOF, ac...",es,League of Legends,24,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-12-02T16:16:51Z,NaN
411,571089346,yeraynayer44,NaN,user,Es un canal en el que se basa en todo tipos de...,es,Far Cry 4,23,https://static-cdn.jtvnw.net/jtv_user_pictures...,2020-08-22T15:42:16Z,NaN
580,574081623,cundoff,NaN,user,NaN,es,Counter-Strike: Global Offensive,4,https://static-cdn.jtvnw.net/jtv_user_pictures...,2020-08-28T03:11:04Z,NaN


In [10]:
from src.data import make_data_from_root_user
from src.data import extract_follows_from_users_df

max_users = 100000
top_users = 5000
initial_user = 'ibai' #Starting point for the random algorithm

make_data_from_root_user(initial_user,output_file="data/data.csv",max_users=max_users)
data = pd.read_csv("data/data.csv")
data.drop_duplicates("id", keep='first',inplace=True) #Remove posible duplicates 
data.fillna({"broadcaster_type":"user"},inplace=True) #Fill nans with 'user' tag
data = data.loc[data['lang']=='es'] #Get just hispanian people
data = data.loc[data['broadcaster_type']!='user'] #Getting rid of not broadcasters/affiliates
data.sort_values(by='view_count', ascending=False,inplace=True) #Sorting by the number of view count

top_df = data.head(top_users).copy()

top_df = extract_follows_from_users_df(top_df)

for i in range(0,top_users):
    data.iloc[i] = top_df.iloc[i]

data.to_csv(r'data/data_davimen.csv',index=False,header=True)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: ibai
INFO:src.data.make_dataset:New users will be fetched until 100000 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 140 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:Iteration 11 - 1354 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv


In [ ]:
data = pd.read_csv("data/data_davimen.csv")
data.head(100)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,NaN,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286989460,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
1,22346597,LVPes,NaN,partner,NaN,es,Special Events,227606609,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,"['572984021', '113366816', '420395016', '23286..."
2,39276140,Rubius,NaN,partner,se contar hasta patata,es,Minecraft,222783415,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12T18:46:56Z,"['277057209', '603776329', '91136321', '198363..."
3,459331509,auronplay,NaN,partner,feliz pero no mucho,es,Minecraft,212157750,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,"['210708721', '431460701', '77649106', '130065..."
4,48878319,TheGrefg,NaN,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Fortnite,197764594,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12T00:45:27Z,"['143776262', '675347177', '33734881', '248222..."
...,...,...,...,...,...,...,...,...,...,...,...
95,73065110,Katth,NaN,partner,¡Bienvenido/a a mi canal! ◕ ‿‿ ◕ ¡Aquí encontr...,es,League of Legends,12871487,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-10-14T19:27:57Z,NaN
96,233398214,TwitchPresentsES,NaN,partner,NaN,es,Twitch Presents,12613722,https://static-cdn.jtvnw.net/jtv_user_pictures...,2018-06-22T21:46:35Z,NaN
97,127443997,NikitonipongoTV,NaN,partner,Streamer de Fortnite para @,es,Fortnite,12425368,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-06-22T14:40:41Z,NaN
98,45329736,HITBOXKING,NaN,partner,Jugador de Valorant desde la Alpha y actualmen...,es,VALORANT,12411129,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-06-28T19:07:15Z,NaN
